<a href="https://colab.research.google.com/github/budd-lab/ML-Model-Selection-and-Cross-Validation/blob/main/AIMLBerkeleyAssignments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SequentialFeatureSelector, SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import set_config
set_config(display="diagram")

import warnings
warnings.filterwarnings('ignore')



In [ ]:
#!mkdir -p ~/.kaggle
#!cp kaggle.json ~/.kaggle/
#!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

ref                                                            title                                            size  lastUpdated          downloadCount  voteCount  usabilityRating  
-------------------------------------------------------------  ----------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
muhammadehsan000/healthcare-dataset-2019-2024                  Healthcare Dataset (2019-2024)                    3MB  2024-08-09 17:52:25           3935         87  1.0              
berkayalan/paris-2024-olympics-medals                          Paris 2024 Olympics Medals                        1KB  2024-08-14 11:02:45           1827         39  1.0              
muhammadehsan000/diabetes-healthcare-dataset                   Diabetes Healthcare Dataset                      27KB  2024-08-17 19:30:34            471         26  1.0              
muhammadehsan000/olympic-games-medal-dataset-1994-2024         Olympic Games Medal Da

In [3]:
# Download the dataset using the Kaggle API (assuming you have kaggle.json set up)
!kaggle datasets download -d kumarajarshi/life-expectancy-who

# Unzip the downloaded file
!unzip life-expectancy-who.zip

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('Life Expectancy Data.csv')

# Display the first few rows to verify the import
print(df.head())

Dataset URL: https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who
License(s): other
  0% 0.00/119k [00:00<?, ?B/s]
100% 119k/119k [00:00<00:00, 36.9MB/s]
Archive:  life-expectancy-who.zip
  inflating: Life Expectancy Data.csv  
       Country  Year      Status  Life expectancy   Adult Mortality  \
0  Afghanistan  2015  Developing              65.0            263.0   
1  Afghanistan  2014  Developing              59.9            271.0   
2  Afghanistan  2013  Developing              59.9            268.0   
3  Afghanistan  2012  Developing              59.5            272.0   
4  Afghanistan  2011  Developing              59.2            275.0   

   infant deaths  Alcohol  percentage expenditure  Hepatitis B  Measles   ...  \
0             62     0.01               71.279624         65.0      1154  ...   
1             64     0.01               73.523582         62.0       492  ...   
2             66     0.01               73.219243         64.0       430  ...   
3        

In [4]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          2938 non-null   object 
 1   Year                             2938 non-null   int64  
 2   Status                           2938 non-null   object 
 3   Life expectancy                  2928 non-null   float64
 4   Adult Mortality                  2928 non-null   float64
 5   infant deaths                    2938 non-null   int64  
 6   Alcohol                          2744 non-null   float64
 7   percentage expenditure           2938 non-null   float64
 8   Hepatitis B                      2385 non-null   float64
 9   Measles                          2938 non-null   int64  
 10   BMI                             2904 non-null   float64
 11  under-five deaths                2938 non-null   int64  
 12  Polio               

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [10]:
# Drop rows with NA values
df=df.dropna()
df
df.describe()

,Year,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
count,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1649.000000,1.649000e+03,1649.000000,1649.000000,1649.000000,1649.000000
mean,2007.840509,69.302304,168.215282,32.553062,4.533196,698.973558,79.217708,2224.494239,38.128623,44.220133,83.564585,5.955925,84.155246,1.983869,5566.031887,1.465363e+07,4.850637,4.907762,0.631551,12.119891
std,4.087711,8.796834,125.310417,120.847190,4.029189,1759.229336,25.604664,10085.802019,19.754249,162.897999,22.450557,2.299385,21.579193,6.032360,11475.900117,7.046039e+07,4.599228,4.653757,0.183089,2.795388
min,2000.000000,44.000000,1.000000,0.000000,0.010000,0.000000,2.000000,0.000000,2.000000,0.000000,3.000000,0.740000,2.000000,0.100000,1.681350,3.400000e+01,0.100000,0.100000,0.000000,4.200000
25%,2005.000000,64.400000,77.000000,1.000000,0.810000,37.438577,74.000000,0.000000,19.500000,1.000000,81.000000,4.410000,82.000000,0.100000,462.149650,1.918970e+05,1.600000,1.700000,0.509000,10.300000
50%,2008.000000,71.700000,148.000000,3.000000,3.790000,145.102253,89.000000,15.000000,43.700000,4.000000,93.000000,5.840000,92.000000,0.100000,1592.572182,1.419631e+06,3.000000,3.200000,0.673000,12.300000
75%,2011.000000,75.000000,227.000000,22.000000,7.340000,509.389994,96.000000,373.000000,55.800000,29.000000,97.000000,7.470000,97.000000,0.700000,4718.512910,7.658972e+06,7.100000,7.100000,0.751000,14.000000
max,2015.000000,89.000000,723.000000,1600.000000,17.870000,18961.348600,99.000000,131441.000000,77.100000,2100.000000,99.000000,14.390000,99.000000,50.600000,119172.741800,1.293859e+09,27.200000,28.200000,0.936000,20.700000


In [11]:
# Assign X and y variables for model fitting

X=df.drop(columns=['Life expectancy ']).select_dtypes(include='number')
y=df['Life expectancy ']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)



In [12]:
# Assign Pipeline for model fitting

pipe_seq = Pipeline([
    ('polyfeatures',PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler',StandardScaler()),
    ('seq',SequentialFeatureSelector(estimator=LinearRegression(),n_features_to_select=5)),
    ('linreg',LinearRegression())
])

pipe_lasso = Pipeline([
    ('polyfeatures',PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler',StandardScaler()),
    ('lasso',Lasso(random_state=42))
])

pipe_ridge = Pipeline([
    ('polyfeatures',PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler',StandardScaler()),
    ('ridge',Ridge(random_state=42))
])


pipe_seq.fit(X_train,y_train)
pipe_lasso.fit(X_train,y_train)
pipe_ridge.fit(X_train,y_train)


seq_coef = pipe_seq.named_steps['linreg'].coef_
lasso_coef = pipe_lasso.named_steps['lasso'].coef_
ridge_coef = pipe_ridge.named_steps['ridge'].coef_

print("Seq Coef:", seq_coef)
print("Lasso Coef:", lasso_coef)
print("Ridge Coef:", ridge_coef)

print("Seq Score:", pipe_seq.score(X_test,y_test))
print("Lasso Score:", pipe_lasso.score(X_test,y_test))
print("Ridge Score:", pipe_ridge.score(X_test,y_test))

print("Seq MSE:", mean_squared_error(y_test,pipe_seq.predict(X_test)))
print("Lasso MSE:", mean_squared_error(y_test,pipe_lasso.predict(X_test)))
print("Ridge MSE:", mean_squared_error(y_test,pipe_ridge.predict(X_test)))

Seq Coef: [-4.23681279 -4.64585137 -5.08622869  4.82388459  9.15712711]
Lasso Coef: [-0.         -0.         -0.          0.          0.          0.
 -0.          0.         -0.          0.          0.          0.
 -0.          0.         -0.         -0.         -0.          0.
  1.30533624 -0.         -1.10497713 -0.          0.          0.
  0.         -0.          0.         -0.          0.          0.
  0.         -1.54621818  0.         -0.         -0.         -0.
  0.          0.         -0.95716838 -0.         -0.          0.
 -0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.          0.         -0.         -0.         -0.
 -0.         -0.         -0.         -0.         -0.         -0.
 -0.         -0.         -0.         -0.         -0.         -0.
 -0.27566985 -0.         -0.         -0.         -0.         -0.
 -0.          0.          0.          0.         -0.          0.
 -0.          0.          0.          0.         -0.          0.
 -0.  

In [13]:
model=pipe_ridge.named_steps['ridge']
model

Ridge(random_state=42)

In [19]:
from sklearn.model_selection import GridSearchCV, train_test_split, KFold, LeaveOneOut, PredefinedSplit

param_grid = {'alpha': [0.1, 1.0, 10.0]}

# 1. Simulating Holdout Cross-Validation with GridSearchCV
# We create a predefined split where the test set is used as validation in GridSearchCV

test_indices = [-1] * len(X_test)
train_indices = [0] * len(X_train)
test_fold = test_indices + train_indices # Combine indices for PredefinedSplit
ps = PredefinedSplit(test_fold=test_fold)

grid_search_holdout = GridSearchCV(model, param_grid, cv=ps, scoring='neg_mean_squared_error')
grid_search_holdout.fit(X, y)
holdout_score = grid_search_holdout.best_score_
holdout_best_params = grid_search_holdout.best_params_

# 2. K-Fold Cross-Validation with GridSearchCV
#kf = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search_kfold = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_kfold.fit(X, y)
kfold_score = grid_search_kfold.best_score_
kfold_best_params = grid_search_kfold.best_params_

# 3. Leave-One-Out Cross-Validation with GridSearchCV
grid_search_loo = GridSearchCV(model, param_grid, cv=LeaveOneOut(), scoring='neg_mean_squared_error')
grid_search_loo.fit(X, y)
loo_score = grid_search_loo.best_score_
loo_best_params = grid_search_loo.best_params_

# Display the results
print(f"Holdout CV - Best Score: {holdout_score:.4f}, Best Params: {holdout_best_params}")
print(f"K-Fold CV - Best Score: {kfold_score:.4f}, Best Params: {kfold_best_params}")
print(f"LOO CV - Best Score: {loo_score:.4f}, Best Params: {loo_best_params}")

Holdout CV - Best Score: -54.1318, Best Params: {'alpha': 10.0}
K-Fold CV - Best Score: -15.1707, Best Params: {'alpha': 0.1}
LOO CV - Best Score: -12.8850, Best Params: {'alpha': 0.1}


In [26]:
holdout_mse=-holdout_score
kfold_mse=-kfold_score
loo_mse=-loo_score

print("Holdout MSE:", holdout_mse)
print("K-Fold MSE:", kfold_mse)
print("LOO MSE:", loo_mse)



Holdout MSE: 54.131811724697904
K-Fold MSE: 15.170684507741672
LOO MSE: 12.885032072095992


In [28]:
# Create a DataFrame for the plot
mse_data = {
    'Method': ['Holdout', 'K-Fold', 'LOO'],
    'MSE': [holdout_mse, kfold_mse, loo_mse]
}
mse_df = pd.DataFrame(mse_data)

# Create the bar plot
fig = px.bar(mse_df, x='Method', y='MSE',
             title='Comparison of Cross-Validation Methods using Mean Squared Error',
             labels={'Score': 'Mean Squared Error'},
             text_auto=True)

# Customize the plot appearance
fig.update_traces(marker_color='blue', textposition='outside')

# Show the plot
fig.show()

In [29]:
!apt-get install -y git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [30]:
!git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


/bin/bash: line 1: CV.ipynb: command not found
cp: target 'https://github.com/budd-lab/ML-Model-Selection-and-Cross-Validation/tree/main/Model' is not a directory


In [32]:
!git clone https://github.com/budd-lab/ML-Model-Selection-and-Cross-Validation.git


Cloning into 'ML-Model-Selection-and-Cross-Validation'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [34]:
%cd ML-Model-Selection-and-Cross-Validation

/content/ML-Model-Selection-and-Cross-Validation


In [37]:
!git add .
!git commit -m "Adding notebook"
!git push -u origin master

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
error: src refspec master does not match any
error: failed to push some refs to 'https://github.com/budd-lab/ML-Model-Selection-and-Cross-Validation.git'


In [36]:
!git config --global user.email "buddhendra@google.com"
!git config --global user.name "Buddhendra"

In [38]:
!git init

Reinitialized existing Git repository in /content/ML-Model-Selection-and-Cross-Validation/.git/


In [39]:
!git add .
!git commit -m "Initial commit"

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [40]:
!git remote add origin https://github.com/budd-lab/ML-Model-Selection-and-Cross-Validation.git

error: remote origin already exists.


In [41]:
!git push -u origin master

error: src refspec master does not match any
error: failed to push some refs to 'https://github.com/budd-lab/ML-Model-Selection-and-Cross-Validation.git'


In [45]:
!cp AIMLBerkeleyAssignments.ipynb /content/ML-Model-Selection-and-Cross-Validation

cp: cannot stat 'AIMLBerkeleyAssignments.ipynb': No such file or directory


In [46]:
pwd

'/content/ML-Model-Selection-and-Cross-Validation'